In [4]:
import tensorflow as tf

matrix1 = tf.constant([[3,3]],name="matrix1")
matrix2 = tf.constant([[2],[2]],name="matrix2")
product = tf.matmul(matrix1, matrix2, name="product")

tf.summary.histogram("matrix1", matrix1)
tf.summary.histogram("matrix2", matrix2)
tf.summary.histogram("product", product)

with tf.Session() as sess:
    merged = tf.summary.merge_all()   
    writer = tf.summary.FileWriter("./lec/sun")
    writer.add_graph(sess.graph)  # Show the graph
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
             
    summary, _ = sess.run([merged,product], feed_dict={})    
    writer.add_summary(summary)



In [6]:
import tensorflow as tf
import numpy as np
# from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

# AND
x_data = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]], dtype=np.float32)
y_data = np.array([[0],
                  [0],
                  [0],
                  [1]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

In [7]:
with tf.name_scope("layer") as scope:    
    W = tf.Variable(tf.random_normal([2, 1]), name='weight')
    b = tf.Variable(tf.random_normal([1]), name='bias')

    hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
    
    w_hist = tf.summary.histogram("weight", W)
    b_hist = tf.summary.histogram("bias", b)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

In [8]:
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [9]:
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

In [12]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./lec")
    writer.add_graph(sess.graph)  # Show the graph
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(1001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


Hypothesis:  [[ 0.00877521]
 [ 0.15212096]
 [ 0.15131731]
 [ 0.78323901]] 
Correct:  [[ 0.]
 [ 0.]
 [ 0.]
 [ 1.]] 
Accuracy:  1.0
